In [1]:
import cv2
import numpy as np
from collections import deque

# --- Centroid Tracker ---
class CentroidTracker:
    def __init__(self, max_lost=5):
        self.next_id = 0
        self.objects = {}         # objectID -> centroid
        self.trajectories = {}    # objectID -> deque of centroids
        self.lost = {}            # objectID -> lost count
        self.max_lost = max_lost

    def update(self, detections):
        updated_objects = {}

        # Initialize new objects if nothing tracked yet
        if not self.objects:
            for det in detections:
                self.objects[self.next_id] = det
                self.trajectories[self.next_id] = deque(maxlen=32)
                self.trajectories[self.next_id].append(det)
                self.lost[self.next_id] = 0
                self.next_id += 1
            return self.objects, self.trajectories

        object_ids = list(self.objects.keys())
        object_centroids = list(self.objects.values())

        used_ids = set()

        for det in detections:
            distances = [np.linalg.norm(np.array(det) - np.array(obj)) for obj in object_centroids]
            min_dist_idx = np.argmin(distances)
            min_id = object_ids[min_dist_idx]

            if distances[min_dist_idx] < 50:
                updated_objects[min_id] = det
                self.trajectories[min_id].append(det)
                self.lost[min_id] = 0
                used_ids.add(min_id)

        # Add unmatched detections as new objects
        for det in detections:
            if not any(np.all(det == obj) for obj in updated_objects.values()):
                self.objects[self.next_id] = det
                self.trajectories[self.next_id] = deque(maxlen=32)
                self.trajectories[self.next_id].append(det)
                self.lost[self.next_id] = 0
                self.next_id += 1

        # Update lost counters
        for obj_id in list(self.objects.keys()):
            if obj_id not in updated_objects:
                self.lost[obj_id] += 1
                if self.lost[obj_id] > self.max_lost:
                    del self.objects[obj_id]
                    del self.trajectories[obj_id]
                    del self.lost[obj_id]
            else:
                self.objects[obj_id] = updated_objects[obj_id]

        return self.objects, self.trajectories

# --- Video Setup ---
cap = cv2.VideoCapture("sports.mp4")
fgbg = cv2.createBackgroundSubtractorMOG2(detectShadows=False)
tracker = CentroidTracker()

frame_rate = cap.get(cv2.CAP_PROP_FPS)
frame_time = 1 / frame_rate if frame_rate > 0 else 1 / 30

# --- Main Loop ---
while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, (960, 540))
    fgmask = fgbg.apply(frame)

    contours, _ = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    detections = []

    for cnt in contours:
        if cv2.contourArea(cnt) > 1000:
            x, y, w, h = cv2.boundingRect(cnt)
            center = (x + w // 2, y + h // 2)
            detections.append(center)

    objects, trajectories = tracker.update(detections)

    for obj_id, center in objects.items():
        traj = trajectories[obj_id]

        # Draw trajectory (limit to last 10 points)
        if len(traj) > 1:
            for i in range(1, min(len(traj), 10)):
                cv2.line(frame, traj[-i], traj[-i - 1], (0, 0, 255), 2)

            # Speed calculation (between last two points)
            distance = np.linalg.norm(np.array(traj[-1]) - np.array(traj[-2]))
            if distance > 2:
                speed = distance / frame_time
                cv2.putText(frame, f'ID:{obj_id} {speed:.1f}px/s',
                            (center[0] + 10, center[1]),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)

        # Draw marker & ID
        cv2.circle(frame, center, 6, (0, 255, 0), -1)
        cv2.putText(frame, f'ID:{obj_id}', (center[0] - 15, center[1] - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)

    cv2.imshow("Clean Multi-Person Tracker", frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()
